# Text classification with movie reviews

Notebook này sẽ phân loại các nhận xét phim (chứa trong một tập dữ liệu) vào 1 trong 2 loại "*tích cực*" hoặc "*tiêu cực*". 

Đây là một ví dụ của phân loại nhị phân, hoặc phân loại 2 lớp (*binary—or two-class—classification*), vốn quan trọng và được sử dụng rộng rãi trong machine learning.

Chúng ta sẽ dùng tập dữ liệu của [IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) chứa 50,000 nhận xét về các bộ phim. Tập dữ liệu này được chia làm 2 phần, 25,000 reviews cho training, và 25,000 reviews cho testing. 

Tập training và testing được tạo một cách đồng đều, nghĩa là số lượng nhận xét tích cực và tiêu cực ở hai bên là như nhau.

In [29]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.10.0


## Download the IMDB dataset

The IMDB dataset comes packaged with TensorFlow. It has already been preprocessed such that the reviews (sequences of words) have been converted to sequences of integers, where each integer represents a specific word in a dictionary.

The following code downloads the IMDB dataset to your machine (or uses a cached copy if you've already downloaded it):

In [30]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

The argument num_words=10000 keeps the top 10,000 most frequently occurring words in the training data. The rare words are discarded to keep the size of the data manageable.

## Explore the data

Let's take a moment to understand the format of the data. The dataset comes preprocessed: each example is an array of integers representing the words of the movie review. Each label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

In [31]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


The text of reviews have been converted to integers, where each integer represents a specific word in a dictionary. Here's what the first review looks like:

In [32]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [33]:
len(train_data[0]), len(train_data[1])

(218, 189)

Do độ dài từng câu trong train_data khác nhau, nên khi gọi ```train_data.shape``` sẽ trả về (25000,). Điều này nghĩa là train_data có tổng cộng 25000 câu (dimension thứ 1), còn dimension thứ 2 không xác định (ứng với độ dài từng câu). 

In [34]:
train_data.shape

(25000,)

### Convert the integers back to words

It may be useful to know how to convert integers back to text. Here, we'll create a helper function to query a dictionary object that contains the integer to string mapping:

In [9]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [10]:
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

## Prepare the data

Những reviews - vốn là những array chứa số integers, cần phải được chuyển thành "tensors" trước khi đẩy vào neural network. Việc chuyển đổi có thể thực hiện thông qua một trong những cách sau:

- Sử dụng one-hot-encode để chuyển array thành những vectors của 0 và . Ví dụ: chuỗi [3, 5] sẽ trở thành một vector 10,000 chiều với mọi phần tử đều bằng 0 trừ indices 3 và 5 (bằng 1). Đây sẽ trở thành layer thứ 1 trong network, layer dạng "Dense", có thể xử lý các data dạng "floating point vector". Cách này đòi nhiều bộ nhớ vì cần một ma trận kích thước num_words * num_reviews 
- Một cách khác, ta có thể chèn vào arrays sao cho chúng có đội dài tương tự nhau, sao đó tạo một integer tensor có kích thước max_length x num_reviews. We can use an embedding layer capable of handling this shape as the first layer in our network.

Tutorial này sử dụng cách tiếp cận thứ 2. 

In [23]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value= word_index["<PAD>"],
                                                       padding = 'post',
                                                       maxlen = 256)
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value= word_index["<PAD>"],
                                                       padding = 'post',
                                                       maxlen = 256)

Lúc này mỗi câu trong train_data đều có đủ 256 phần tử (ứng với ký tự PAD). Do vậy khi gọi ```train_data.shape```, thì kết quả trả về (25000, 256) thay vì (25000,)

In [25]:
train_data.shape

(25000, 256)

In [26]:
len(train_data)

25000

In [27]:
len(train_data[0])

256

In [28]:
train_data[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

## Build the model

Neural network được tạo ra nhờ việc ghép nối các layers - nghĩa là cần xác định 2 điều:
- Cần dùng bao nhiêu layer?
- Cần dùng bao nhiêu "hidden unit" với mỗi layer? 

Trong ví dụ này, input data chứa một mảng những word-indices. Label để dự đoán là 0 hoặc 1. 

In [36]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)

vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:

The first layer is an Embedding layer. This layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: (batch, sequence, embedding).
Next, a GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model can handle input of variable length, in the simplest way possible.
This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability, or confidence level.